# Unfinished Draft: 
## Part 1. Managing Order Book States/Messages in PostgreSQL 
## Part 2. Recontructing the Order Book using Data Stored in PostgreSQL
## Future Tutorial: CRUD on order book statistics and visualization with Grafana 

This script is about showing you how to setup postgres access using the crobat package
The advantages to postgres is that its a long term data storage engine
While I need to work on archiving old data, rdms is great for doing panalysis 

In [1]:
# lets try set up the environment.
import os, sys
os.chdir('../..')
#os.chdir('..') # moves us 2 folders back into to allow for imports that users would use
wd = os.getcwd()
print(wd)
sys.path.append(wd)
#print(os.getcwd())
for i in sys.path:
    print(i)

/home/ivan/Documents/github
/home/ivan/Documents/github/crobat/Jupyter nbs
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload

/home/ivan/.local/lib/python3.8/site-packages
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/home/ivan/.local/lib/python3.8/site-packages/IPython/extensions
/home/ivan/.ipython
/home/ivan/Documents/github


In [2]:
# lets try importing the postgres setup 
from crobat.postgres import recorder as rec 
from copra.websocket import Channel, Client

printing the sub sys path for the recorder module for postgres
/home/ivan/Documents/github/crobat/Jupyter nbs
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload

/home/ivan/.local/lib/python3.8/site-packages
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/home/ivan/.local/lib/python3.8/site-packages/IPython/extensions
/home/ivan/.ipython
/home/ivan/Documents/github
/home/ivan/Documents/github/crobat/crobat
/home/ivan/Documents/github/crobat/postgres
the last entry in sys.path is.... /home/ivan/Documents/github/tests


In [3]:
# Step 1 : defining settings class
class input_args(object):
    def __init__(self, currency_pair='ETH-USD', position_range=5, recording_duration=5):
        self.currency_pair = currency_pair
        self.position_range = position_range
        self.recording_duration = recording_duration

settings = input_args()
print(dir(settings))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'currency_pair', 'position_range', 'recording_duration']


In [3]:
# step 2 constructing the ws connection
import asyncio
import nest_asyncio
nest_asyncio.apply()
loop = asyncio.get_event_loop()
channel = Channel('level2', settings.currency_pair) 
#channel2 = Channel('ticker', settings.currency_pair)
ws = rec.Ticker(loop, channel, settings)
#ws.subscribe(channel2)
try:
    loop.run_forever()
except KeyboardInterrupt:
    loop.run_until_complete(ws.close())
    loop.close()

NameError: name 'settings' is not defined

In [3]:
# Lets try fetching the data from postgres
from crobat.postgres import sqlconnection as sqlc

# Step 0 open the sql connection to pick up cursor and connection vars
cursor, connection = sqlc.open_SQL_connection()
# Step 1 setup the read instance

read_instance = sqlc.psql_read_operations(cursor, connection)

tstamp = read_instance.get_last_tstamp()
print(tstamp)

sqlc.close_SQL_connection(cursor, connection)

the last entry in sys.path is.... /home/ivan/Documents/github/tests
Connecting to the PostgreSQL database...
2021-08-19 18:18:36.213520
2021-08-19 18:18:36.213520
closed the cursor
Database connection closed.


In [8]:
# Step 0 open the sql connection to pick up cursor and connection vars
cursor, connection = sqlc.open_SQL_connection()
# Step 1 setup the read instance

read_instance = sqlc.psql_read_operations(cursor, connection)
# lets pick up the appropriate connection id
sql = """ SELECT snapshot_connection_id from snapshots order by tstamp desc limit 1;"""
cid = read_instance.custom_sql_fetch(sql)
print(cid)
# with this lets pick up all the messages that match this CID
sql = """ SELECT * from messages WHERE messages.snapshot_connection_id = '{}' """.format(cid)
messages = read_instance.custom_sql_fetch(sql)

#  Now that we can access the messages lets try to get elements from the snapshot.

sql = """ SELECT json_array_elements(bids) FROM snapshots WHERE snapshot_connection_id = '{}'""".format(cid)
bids = read_instance.custom_sql_fetch(sql)
sql = """ SELECT json_array_elements(asks) FROM snapshots WHERE snapshot_connection_id = '{}'""".format(cid)
asks = read_instance.custom_sql_fetch(sql)
sqlc.close_SQL_connection(cursor, connection)
snapshot_message = {
    'bids' : bids,
    'asks' : asks
}
print(snapshot_message)

Connecting to the PostgreSQL database...
only returning one item
ETH-USD1629411508897421
multi item return of len:  1480
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>